In [51]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import os
from embeddings import get_embeddings
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import polars as pl
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

## no count balance & no relative balance

## count balance

 לא חייבים לעשות באלאנס. רק בגלל שהדאטה עכשיו לא טוב עשיתי באלאנס. 
 ל

In [54]:
column_name = ['noisy', 'abstract', 'paint'][2]

csv_path = '/Users/yuda/PycharmProjects/matching_arts_v3/data/eli_votes.csv'
images_folder = '/Users/yuda/Desktop/vs_code_projects/data/'

# Load and filter
df = pl.read_csv(csv_path)

# Filter to only filled and non-null target
df = df.filter(
    (pl.col("filled") == True) &
    (pl.col(column_name).is_not_null())
)

# Add file paths
df = df.with_columns([
    (pl.col("image_id").cast(str) + ".jpg").alias("file_name"),
    (pl.lit(images_folder) + "/" + pl.col("image_id").cast(str) + ".jpg").alias("file_path")
])

# Convert to lists
X_all = df["file_path"].to_list()
y_all = df[column_name].cast(int).to_list()

# Ensure equal class distribution by balancing the classes
unique_classes = np.unique(y_all)
min_class_count = min([y_all.count(i) for i in unique_classes])  # Find the class with the minimum count

# Create balanced dataset by sampling from each class
X_balanced = []
y_balanced = []

for class_label in unique_classes:
    # Get indices for each class
    indices = [i for i, label in enumerate(y_all) if label == class_label]
    
    # Randomly sample the same number of examples for each class based on the smallest class count
    sampled_indices = np.random.choice(indices, min_class_count, replace=False)
    
    # Add the sampled data to the balanced list
    X_balanced.extend([X_all[i] for i in sampled_indices])
    y_balanced.extend([y_all[i] for i in sampled_indices])

# Now we have a balanced dataset
X_balanced = np.array(X_balanced)
y_balanced = np.array(y_balanced)

# Stratified split of the balanced data
X_train_names, X_test_names, y_train_list, y_test_list = train_test_split(
    X_balanced, y_balanced, train_size=0.8, stratify=y_balanced, random_state=42
)

print(f"Train size: {len(X_train_names)}")
print(f"Test size: {len(X_test_names)}")
print(f"Train class distribution: {dict(zip(*np.unique(y_train_list, return_counts=True)))}")
print(f"Test class distribution: {dict(zip(*np.unique(y_test_list, return_counts=True)))}")


Train size: 184
Test size: 46
Train class distribution: {1: 37, 2: 37, 3: 37, 4: 36, 5: 37}
Test class distribution: {1: 9, 2: 9, 3: 9, 4: 10, 5: 9}


In [55]:
X_train_list = get_embeddings(X_train_names)
X_test_list = get_embeddings(X_test_names)

In [56]:
X_train: np.ndarray = np.array(X_train_list)
y_train: np.ndarray = np.array(y_train_list) / 5

X_test: np.ndarray = np.array(X_test_list)
y_test: np.ndarray = np.array(y_test_list) / 5

In [57]:
# clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf = LinearRegression().fit(X_train, y_train)
predictions = clf.predict(X_test)
predictions

array([0.69023403, 0.15915575, 0.09881608, 0.71176059, 0.89167015,
       0.19567376, 0.68265832, 0.58352681, 0.63813604, 0.8922714 ,
       0.28882555, 0.42838461, 0.58100467, 0.71838065, 0.71730122,
       0.81083011, 0.45227434, 0.89837141, 0.39629056, 0.871421  ,
       0.44338541, 0.52141948, 0.89323791, 0.55612716, 0.58379694,
       0.95920925, 0.35462178, 0.98651435, 0.42957192, 0.74636115,
       0.73664287, 1.04556817, 0.49544142, 0.35438148, 0.06143322,
       0.70954213, 0.73903554, 0.35836353, 0.84287307, 0.47138616,
       0.93453238, 0.63383883, 0.9829853 , 0.42377184, 1.        ,
       0.83410124])

In [58]:
train_predictions = clf.predict(X_train)
train_predictions = np.where(train_predictions < 0.5, 0, 1)
y_train_binary = np.where(y_train < 0.5, 0, 1)

accuracy = accuracy_score(y_train_binary, train_predictions)
print("Train Accuracy:", accuracy)

Train Accuracy: 1.0


In [59]:
test_predictions = clf.predict(X_test)
test_predictions = np.where(test_predictions < 0.5, 0, 1)
y_test_binary = np.where(y_test < 0.5, 0, 1)

accuracy = accuracy_score(y_test_binary, test_predictions)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.6956521739130435


In [60]:
with open('/Users/yuda/PycharmProjects/matching_arts_v3/data/linear_regression_paint.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [49]:

def show_images(images_list, prediction, target):
    """
    Display images with their predictions and target labels.
    Supports both image file paths and numpy arrays.
    """

    num_images = len(images_list)
    cols = min(5, num_images)
    rows = (num_images // cols) + (num_images % cols > 0)

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = axes.flatten() if num_images > 1 else [axes]

    for i in range(num_images):
        # Load image if it's a file path (string)
        img = Image.open(images_list[i]) if isinstance(images_list[i], str) else images_list[i]
        img = np.array(img)  # Convert to NumPy array for `imshow`

        axes[i].imshow(img, cmap="gray" if len(img.shape) == 2 else None)
        axes[i].axis("off")
        axes[i].set_title(f"Pred: {prediction[i]}\nTarget: {target[i]}", fontsize=10)

    plt.tight_layout()
    plt.show()

In [ ]:
show_images(X_test_names[:10], list(predictions)[:10], y_test_list[:10])

In [10]:
# 0.47, 0.64
# 0.63, 0.68
# 0.52, 0.56